In [1]:
from transformers import pipeline
from tqdm import tqdm

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # True if GPU is detected


2.6.0+cpu
False


In [3]:
TOPICS = [
    "Abortion", "Gun Control", "Healthcare", "Immigration", "Climate Change",
    "Taxes", "LGBTQ Rights", "Education", "Police Reform", "Foreign Policy",
    "Voting Rights", "Free Speech", "Welfare", "Student Loans", "Inflation",
    "Unemployment", "Minimum Wage", "Social Security", "Energy Policy", "Criminal Justice"
]

In [4]:
# Load zero-shot classifier once
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

c:\Users\judem\OneDrive\USF\Spring 2025\Software Engineering\social-media-mining-project\env\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\judem\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [5]:
# Classify a single tweet
def classify_political_topic(tweet_text, candidate_labels=TOPICS, threshold=0.4):
    if not isinstance(tweet_text, str) or not tweet_text.strip():
        return "Non-Political"

    result = classifier(tweet_text, candidate_labels=candidate_labels, multi_label=False)
    top_label = result['labels'][0]
    top_score = result['scores'][0]

    return top_label if top_score >= threshold else "Non-Political"

In [6]:
def add_political_labels(df_cleaned, show_progress=False):
    if show_progress:
        from tqdm import tqdm
        tqdm.pandas(desc="Classifying tweets")
        df_cleaned["political_label"] = df_cleaned["clean_tweet"].progress_apply(classify_political_topic)
    else:
        df_cleaned["political_label"] = df_cleaned["clean_tweet"].apply(classify_political_topic)
    return df_cleaned

In [7]:
import pandas as pd

df_cleaned = pd.read_csv("../data/cleaned_tweets.csv")
new_df = add_political_labels(df_cleaned, show_progress=True)
new_df.to_csv("../data/cleaned_tweets_with_labels.csv", index=False)
new_df.head()

Classifying tweets:   0%|          | 177/404205 [21:12<806:59:57,  7.19s/it]


KeyboardInterrupt: 